In [ ]:
import pandas as pd
import csv
import re
from num2words import num2words
from collections import OrderedDict

In [ ]:
new_types = pd.read_csv('./Feng.tsv', sep='\t')

In [ ]:
# Make a dictionary with key - column header & value = template specification (first row of table).

template_seed = OrderedDict([ ('ID' , 'ID'), ('CLASS_TYPE' , 'CLASS_TYPE'),\
                             ('RDF_Type' , 'TYPE' )])

#label, definition, short synonym:
template_seed.update([("Name" , "A rdfs:label"), ("definition" , "A IAO:0000115"),\
                      ("Xref_def" , ">A oboInOwl:hasDbXref"),\
                      ("created_by" , "A oboInOwl:created_by"),\
                      ("creation_date", "A oboInOwl:creation_date")])

#short name synonym
template_seed.update([("MFSynonym" , "A oboInOwl:hasExactSynonym"),\
                      ("MFsyn_ref" , ">A oboInOwl:hasDbXref"),\
                      ("MTSynonym" , "A oboInOwl:hasNarrowSynonym SPLIT=|"),\
                      ("MTsyn_ref" , ">A oboInOwl:hasDbXref"),\
                      ("comment" , "A rdfs:comment")])

# Columns for relationships:
template_seed.update([("soma" , "SC 'has soma location' some %"),\
                      ("adult_neuron" , "SC %"),\
                      ("ascending", "SC %")])

# Create dataFrame for template
template = pd.DataFrame.from_records([template_seed])

template

In [ ]:
count = 0 # first row
id_mapping = {} # dictionary of ids for types

for i in new_types.index:

    row_od = OrderedDict([]) #new template row as an empty ordered dictionary
    for c in template.columns: #make columns and blank data for new template row
        row_od.update([(c , "")])
    
    #these are the same in each row
    row_od["CLASS_TYPE"] = "subclass"
    row_od["RDF_Type"] = "owl:Class"
    row_od["Xref_def"] = "FlyBase:FBrf0247391"
    row_od["MFsyn_ref"] = "FlyBase:FBrf0247391"
    row_od["MTsyn_ref"] = "FlyBase:FBrf0247391"
    row_od["created_by"] = "http://orcid.org/0000-0002-1373-1705"
    row_od["creation_date"] = "2021-03-17T12:00:00Z"
    row_od["definition"] = '.'
    row_od["comment"] = "Nomenclature scheme will be explained in a later publication (Feng et al., 2020)."

    #easy to generate data
    row_od["ID"] = id_advance(start_id, count)
    row_od["MFSynonym"] = "adult MF%s neuron" % new_types['MF'][i].zfill(2)
    row_od["MTSynonym"] = "adult %s neuron" % new_types['MF'][i]
    row_od["Name"] = "adult %s neuron of ventral nerve cord" % new_types['Cell Type'][i]
    if new_types['ascending'][i] == 1:
        row_od["ascending"] = "FBbt:00048301" # 'adult ascending neuron'
    else:
        row_od["adult_neuron"] = "FBbt:00047095"
    
    row_od["soma"] = 
    
    #make new row into a DataFrame and add it to template
    new_row = pd.DataFrame.from_records([row_od])
    template = pd.concat([template, new_row], ignore_index=True, sort=False)

    count +=1
    
    id_mapping[new_types[i]] = row_od["ID"] # add to dictionary
    
#template